In [1]:
import pandas as pd
import requests
import numpy as np
from bs4 import BeautifulSoup

import re
import camelot
import ssl
ssl._create_default_https_context = ssl._create_unverified_context

In [2]:
#Read dataset as of previous date from the github link
old_df=pd.read_csv("https://raw.githubusercontent.com/Suhailhassanbhat/Covid_Data_Scraper/main/data/master_cases_deaths.csv")

#convert datatype for all numbers. it is important for merging it with the latest data
old_df[['tot_cases','tot_cases_conf','tot_cases_prob', 'tot_deaths', 
            'tot_deaths_conf','tot_deaths_prob', 'new_cases', 'new_deaths',
            'new_cases_conf', 'new_cases_prob', 'new_deaths_conf', 'new_deaths_prob', 
            'new_cases_18_and_under','new_cases_18_and_under_conf', 
            'new_cases_18_and_under_prob','actual_deaths']]=old_df[['tot_cases','tot_cases_conf','tot_cases_prob', 'tot_deaths', 
            'tot_deaths_conf','tot_deaths_prob', 'new_cases', 'new_deaths',
            'new_cases_conf', 'new_cases_prob', 'new_deaths_conf', 'new_deaths_prob', 
            'new_cases_18_and_under','new_cases_18_and_under_conf', 
            'new_cases_18_and_under_prob','actual_deaths']].apply(pd.to_numeric, errors = 'coerce')
#convert date from string to datetime
old_df.date=pd.to_datetime(old_df.date)



In [3]:
# change this to run for other months
month="Aug." 
# read monthly covid page
response=requests.get(f"https://chfs.ky.gov/Pages/cvdaily.aspx?View={month}%202021%20Daily%20Summaries&Title=Table%20Viewer%20Webpart", verify=False)
#create an empty list for links
link_list=[]
#run this code to grab all the links on the page
doc=BeautifulSoup(response.text, 'html.parser')
container=doc.select_one('tbody').select('tr')
for element in container:
    link_dict={}
    for ele in element.find_all('a'): 
        try: 
            link_dict['date']=ele.text.replace("COVID-19 Daily Report", "")
            link_dict['link']=ele.get('href')
        except:
            pass
    link_list.append(link_dict)
latest_df=pd.DataFrame(link_list)
latest_df.date=pd.to_datetime(latest_df.date)
latest_df=latest_df.sort_values('date', ascending=False).reset_index(drop=True)
#we need only first three
latest_df=latest_df[:3]

/Library/Frameworks/Python.framework/Versions/3.8/lib/python3.8/site-packages/urllib3/connectionpool.py:981: InsecureRequestWarning: Unverified HTTPS request is being made to host 'chfs.ky.gov'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  warnings.warn(


In [4]:
latest_df

,date,link
0,2021-08-30,https://chfs.ky.gov/cvdaily/COVID19DailyReport...
1,2021-08-29,https://chfs.ky.gov/cvdaily/COVID19DailyReport...
2,2021-08-28,https://chfs.ky.gov/cvdaily/COVID19DailyReport...


In [5]:
# this code is to read three links and grab all data from the first table
latest_list=[]
#go through each of the three links
for link in latest_df.link:
    #read all tables on page 1
    tables=camelot.read_pdf(link, flavor='lattice', pages='1', verify=False)
    #grab the first table
    cases_summary=tables[0].df.T
    #promote first row to header
    new_header=cases_summary.iloc[0]
    #remove header
    cases_summary=cases_summary[1:]
    cases_summary.columns=new_header
    #reset_index
    cases_summary = cases_summary.reset_index(drop=True)
    cases_sum = cases_summary.drop(cases_summary.columns[0], axis=1)
    #create column names
    category =[]
    category.append(cases_summary.columns[0].split()[1])
    category.append(cases_summary.columns[0].split()[2])
    category.append(cases_summary.columns[0].split()[3])
    categories=pd.DataFrame(category)
    categories.columns=['type']
    cases = categories.join(cases_sum)
    #extract report date and convert it to datetime
    cases['date'] = link.split("Report")[1].replace(".pdf", "").replace("-COVID19","")
    cases['date'] = pd.to_datetime(cases['date'], format='%m%d%y')
    #remove comma separator
    cases=cases.replace(',', '', regex=True)
    #convert to integers in one go
    latest_list.append(cases)
top_df=pd.concat(latest_list)
#------This code converts rows into columns. It is important for the join
cases1=top_df[top_df.type == 'Total'].rename(columns={
    "Cases":"tot_cases", "Deaths":"tot_deaths", "New Cases":"new_cases", "Total New Deaths":"new_deaths", 
    "New Cases 18 and Under":"new_cases_18_and_under"
}).drop(columns=['type']).reset_index(drop=True)

cases2=top_df[top_df.type == 'Confirmed'].rename(columns={
    "Cases":"tot_cases_conf", "Deaths":"tot_deaths_conf", "New Cases":"new_cases_conf", 
    "Total New Deaths":"new_deaths_conf", "New Cases 18 and Under":"new_cases_18_and_under_conf"
}).drop(columns=['type']).reset_index(drop=True)

cases3=top_df[top_df.type == 'Probable'].rename(columns={
    "Cases":"tot_cases_prob", "Deaths":"tot_deaths_prob", "New Cases":"new_cases_prob", 
    "Total New Deaths":"new_deaths_prob"
    ,"New Cases 18 and Under":"new_cases_18_and_under_prob"
}).drop(columns=['type']).reset_index(drop=True)

#concat these three datasets
cases_final=pd.concat([cases1, cases2, cases3], axis=1)
#drop common date column but keep the last one and reset index
cases_final=cases_final.T.reset_index().drop_duplicates().T.reset_index(drop=True)
new_header = cases_final.iloc[0] 
cases_final = cases_final[1:] 
cases_final.columns = new_header
cases_final.date=pd.to_datetime(cases_final.date, format='%Y-%m-%d')
#rearrange columns
cases_final=cases_final[['date','tot_cases','tot_cases_conf','tot_cases_prob', 'tot_deaths', 
            'tot_deaths_conf','tot_deaths_prob', 'new_cases', 'new_deaths',
            'new_cases_conf', 'new_cases_prob', 'new_deaths_conf', 'new_deaths_prob', 
            'new_cases_18_and_under','new_cases_18_and_under_conf', 
            'new_cases_18_and_under_prob']].reset_index(drop=True)
#change datatypes here
cases_final[['tot_cases','tot_cases_conf','tot_cases_prob', 'tot_deaths', 
            'tot_deaths_conf','tot_deaths_prob', 'new_cases', 'new_deaths',
            'new_cases_conf', 'new_cases_prob', 'new_deaths_conf', 'new_deaths_prob', 
            'new_cases_18_and_under','new_cases_18_and_under_conf', 
            'new_cases_18_and_under_prob']]=cases_final[['tot_cases','tot_cases_conf','tot_cases_prob', 'tot_deaths', 
            'tot_deaths_conf','tot_deaths_prob', 'new_cases', 'new_deaths',
            'new_cases_conf', 'new_cases_prob', 'new_deaths_conf', 'new_deaths_prob', 
            'new_cases_18_and_under','new_cases_18_and_under_conf', 
            'new_cases_18_and_under_prob']].apply(pd.to_numeric, errors = 'coerce')

#merge old data with the newly scraped one
final_df=pd.concat([old_df, cases_final]).reset_index(drop=True)
#fill actual deaths column with new deaths
final_df.actual_deaths=final_df.actual_deaths.fillna(final_df.new_deaths)
#drop duplicates
final_df=final_df.drop_duplicates()
#sort by date so that the latest date is at the top
final_df=final_df.sort_values("date", ascending=False)
# final_df.to_csv("master_cases_deaths.csv", index=False)
final_df


,date,tot_cases,tot_cases_conf,tot_cases_prob,tot_deaths,tot_deaths_conf,tot_deaths_prob,new_cases,new_deaths,audit_deaths,new_cases_conf,new_cases_prob,new_deaths_conf,new_deaths_prob,new_cases_18_and_under,new_cases_18_and_under_conf,new_cases_18_and_under_prob,actual_deaths
540,2021-08-30,572517,422922.0,149595.0,7741.0,6933.0,808.0,2619.0,25.0,NaN,2072.0,547.0,21.0,4.0,752.0,577.0,175.0,25.0
541,2021-08-29,569903,420843.0,149060.0,7716.0,6912.0,804.0,3249.0,18.0,NaN,2573.0,676.0,17.0,1.0,946.0,742.0,204.0,18.0
542,2021-08-28,566667,418271.0,148396.0,7698.0,6895.0,803.0,4050.0,13.0,NaN,2775.0,1275.0,12.0,1.0,1285.0,872.0,413.0,13.0
0,2021-08-27,562623,415518.0,147105.0,7685.0,6883.0,802.0,4815.0,18.0,NaN,3409.0,1406.0,13.0,5.0,1529.0,1010.0,519.0,18.0
1,2021-08-26,557835,412143.0,145692.0,7667.0,6870.0,797.0,5401.0,27.0,NaN,3872.0,1529.0,22.0,5.0,1759.0,1198.0,561.0,27.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
535,2020-03-10,6,NaN,NaN,NaN,NaN,NaN,2.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0
536,2020-03-09,4,NaN,NaN,NaN,NaN,NaN,3.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0
537,2020-03-08,1,NaN,NaN,NaN,NaN,NaN,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0
538,2020-03-07,1,NaN,NaN,NaN,NaN,NaN,1.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0


In [21]:
# missing_data=pd.DataFrame([{"date":"2021-08-20","tot_cases":534430,"tot_cases_conf":395654,"tot_cases_prob":138776,"tot_deaths":7517,"tot_deaths_conf":6739,"tot_deaths_prob":778,"new_cases":3869,"new_deaths":6,"audit_deaths":34,"new_cases_conf":2815,"new_cases_prob":1054,"new_deaths_conf":3,"new_deaths_prob":3,"new_cases_18_and_under":1158,"new_cases_18_and_under_conf":804,"new_cases_18_and_under_prob":354,"actual_deaths":6}])

# missing_data.date=pd.to_datetime(missing_data.date, format='%Y-%m-%d')
# final_df=pd.concat([final_df,missing_data]).reset_index(drop=True).sort_values("date", ascending=False)


# Seven day average calculations and weekly and biweekly changes

In [20]:
#filter master cases and deaths file
daily_df=final_df[['date','new_cases', 'new_deaths', 'actual_deaths']]

#sort by date in ascending order;important for average calculations
sorted_df=daily_df.sort_values('date')

#seven day averages
sorted_df['avg_cases']=sorted_df.new_cases.rolling(7).mean().round()
sorted_df['avg_deaths']=sorted_df.new_deaths.rolling(7).mean().round()
sorted_df['avg_actual_deaths']=sorted_df.actual_deaths.rolling(7).mean().round()

#per 100k calculations
ky_population=4467673 # population according to census bureau estimates 2019
sorted_df['cases_per_100k']=(sorted_df.new_cases *100000/ky_population).round()
sorted_df['avg_cases_per_100k']=(sorted_df.avg_cases *100000/ky_population).round()

#weekly  changes
sorted_df['pct_chng_cases_weekly']=(sorted_df.avg_cases.pct_change(periods=7)*100).round()
sorted_df['pct_chng_deaths_weekly']=(sorted_df.avg_deaths.pct_change(periods=7)*100).round()
sorted_df['pct_chng_adeaths_weekly']=(sorted_df.avg_actual_deaths.pct_change(periods=7)*100).round()

#biweekly changes
sorted_df['pct_chng_cases_biweekly']=(sorted_df.avg_cases.pct_change(periods=14)*100).round()
sorted_df['pct_chng_deaths_biweekly']=(sorted_df.avg_deaths.pct_change(periods=14)*100).round()
sorted_df['pct_chng_adeaths_biweekly']=(sorted_df.avg_actual_deaths.pct_change(periods=14)*100).round()

#sort it back to the original order
resorted_df=sorted_df.sort_values('date', ascending=False)
#convert infinity values to nan values
resorted_df=resorted_df.replace(np.inf, np.nan)
resorted_df.to_csv("daily_report.csv", index=False)

# Testing scraper

In [22]:
github_url="https://raw.githubusercontent.com/Suhailhassanbhat/Covid_Data_Scraper/main/data/master_testing.csv"
test_df=pd.read_csv(github_url)
test_df.date=pd.to_datetime(test_df.date)
test_df[['tot_tests', 'total_pcr_tests', 'total_serology_tests',
       'total_antigen_tests', 'total_positive_tests', 'total_pcr_positive',
       'total_serology_positive', 'total_antigen_positive']]=test_df[['tot_tests', 'total_pcr_tests', 'total_serology_tests',
       'total_antigen_tests', 'total_positive_tests', 'total_pcr_positive',
       'total_serology_positive', 'total_antigen_positive']].apply(pd.to_numeric, errors = 'coerce')


,date,tot_tests,total_pcr_tests,total_serology_tests,total_antigen_tests,total_positive_tests,total_pcr_positive,total_serology_positive,total_antigen_positive
0,2021-08-23,7758850.0,6038093,362649.0,1197516.0,663944.0,472458.0,41458.0,150028.0
1,2021-08-22,7736183.0,6020622,361850.0,1193493.0,660741.0,470071.0,41193.0,149477.0
2,2021-08-21,7711207.0,6002647,360971.0,1187801.0,657111.0,467444.0,40966.0,148701.0
3,2021-08-19,7646488.0,5958963,359005.0,1170779.0,648436.0,461535.0,40434.0,146467.0
4,2021-08-18,7606685.0,5931984,358052.0,1160139.0,643162.0,457855.0,40196.0,145111.0


In [8]:
test_list=[]
links=latest_df.link
for link in links:
    test_dict={}
    tables=camelot.read_pdf(link, flavor='stream',table_areas=['37,121,580,60'], split_text=True)
    # tables=camelot.read_pdf(link, flavor='stream',table_areas=['37,151,580,60'], split_text=True)

    test_table=tables[0].df.iloc[3].str.replace(",", "")
    test_dict['date']=link.split("Report")[1].replace(".pdf", "").replace("-COVID19","")
    test_dict["tot_tests"]=test_table[0]
    test_dict['total_pcr_tests']=test_table[1]
    test_dict['total_serology_tests'] =test_table[2]
    test_dict['total_antigen_tests']=test_table[3]
    test_dict['total_positive_tests']=test_table[4]
    test_dict['total_pcr_positive']=test_table[5]
    test_dict['total_serology_positive']=test_table[6]
    test_dict['total_antigen_positive']=test_table[7]
    test_list.append(test_dict)
latest_test_df=pd.DataFrame(test_list)
latest_test_df.date=pd.to_datetime(latest_test_df.date, format='%m%d%y')

latest_test_df[['tot_tests', 'total_pcr_tests', 'total_serology_tests',
       'total_antigen_tests', 'total_positive_tests', 'total_pcr_positive',
       'total_serology_positive', 'total_antigen_positive']]=latest_test_df[['tot_tests', 'total_pcr_tests', 'total_serology_tests',
       'total_antigen_tests', 'total_positive_tests', 'total_pcr_positive',
       'total_serology_positive', 'total_antigen_positive']].apply(pd.to_numeric, errors = 'coerce')

#merge old data with the newly scraped one
final_test_df=pd.concat([test_df, latest_test_df]).reset_index(drop=True)
#drop duplicates
final_test_df=final_test_df.drop_duplicates()
#sort by date so that the latest date is at the top
final_test_df=final_test_df.sort_values("date", ascending=False)
final_test_df.to_csv("master_testing.csv", index=False)

In [40]:
daily_tests=final_test_df[["date", "total_pcr_tests", "total_pcr_positive"]]
daily_tests=daily_tests.sort_values("date", ascending=True)

#calculate daily values
daily_tests["daily_pcr"]=daily_tests.total_pcr_tests.diff()
daily_tests["daily_pcr_positive"]=daily_tests.total_pcr_positive.diff()
#per 100k calculations
daily_tests['pcr_per_100k']=(daily_tests.daily_pcr *100000/ky_population).round()
daily_tests['positive_pcr_per_100k']=(daily_tests.daily_pcr_positive *100000/ky_population).round()

#weekly averages
daily_tests['avg_pcr']=daily_tests.daily_pcr.rolling(7).mean().round()
daily_tests['avg_pcr_positive']=daily_tests.daily_pcr_positive.rolling(7).mean().round()
daily_tests['avg_pcr_per_100k']=daily_tests.pcr_per_100k.rolling(7).mean().round()
daily_tests['avg_positive_pcr_per_100k']=daily_tests.positive_pcr_per_100k.rolling(7).mean().round()

#positivity rate calculations
daily_tests['daily_positive_rate']=(daily_tests.daily_pcr_positive*100/daily_tests.daily_pcr).round()
daily_tests['avg_positive_rate']=(daily_tests.avg_pcr_positive*100/daily_tests.avg_pcr).round()

daily_tests_resorted=daily_tests.sort_values('date', ascending=False)

daily_tests_resorted.to_csv("daily_testing.csv", index=False)

# Hospitalizations, ICUS and ventilators

In [6]:
hosp_url="https://raw.githubusercontent.com/Suhailhassanbhat/Covid_Data_Scraper/main/data/hospitalization_data.csv"
hosp_df=pd.read_csv(hosp_url)
hosp_df.date=pd.to_datetime(hosp_df.date)

hosp_df[['hospitalized', 'in_icu', 'on_vent']]=hosp_df[[
    'hospitalized', 'in_icu', 'on_vent']].apply(pd.to_numeric, errors = 'coerce')


In [10]:
hosp_list=[]
links=latest_df.link
for link in links:
    hosp_dict={}
    tables=camelot.read_pdf(link, flavor='lattice', pages='2')
    hosp_table=tables[1].df
    hosp_dict["date"]=link.split("Report")[1].replace(".pdf", "").replace("-COVID19","")
    hosp_dict["hospitalized"]=hosp_table.iloc[1][1].replace(",", "")
    hosp_dict["in_icu"]=hosp_table.iloc[2][1]
    hosp_dict["on_vent"]=hosp_table.iloc[3][1]

    hosp_list.append(hosp_dict)
latest_hosp_df=pd.DataFrame(hosp_list)
latest_hosp_df.date=pd.to_datetime(latest_hosp_df.date, format='%m%d%y')
latest_hosp_df[['hospitalized', 'in_icu', 'on_vent']]=latest_hosp_df[['hospitalized', 'in_icu', 'on_vent']].apply(pd.to_numeric, errors = 'coerce')

final_hosp_df=pd.concat([hosp_df,latest_hosp_df]).reset_index(drop=True)
final_hosp_df=final_hosp_df.drop_duplicates()
final_hosp_df.date=pd.to_datetime(final_hosp_df.date, format='%Y-%m-%d')
final_hosp_df=final_hosp_df.sort_values('date', ascending=False).reset_index(drop=True)
final_hosp_df

,date,hospitalized,in_icu,on_vent
0,2021-08-30,2198,615,384.0
1,2021-08-29,2137,610,361.0
2,2021-08-28,2173,596,346.0
3,2021-08-27,2129,592,349.0
4,2021-08-26,2115,590,345.0
...,...,...,...,...
501,2020-04-16,412,252,NaN
502,2020-04-15,305,137,NaN
503,2020-04-14,299,136,NaN
504,2020-04-13,289,136,NaN


In [54]:
final_hosp_df.to_csv("hospitalization_data.csv", index=False)

In [44]:
hosp_list=[]
links=latest_df.link
for link in links:
    hosp_dict={}
    tables=camelot.read_pdf(link, flavor='lattice', pages='2')
    hosp_table=tables[2].df
    hosp_dict["date"]=link.split("Report")[1].replace(".pdf", "").replace("-COVID19","")
    hosp_dict["hospitalized"]=hosp_table.iloc[1][1].replace(",", "")
    hosp_dict["in_icu"]=hosp_table.iloc[2][1].replace(",", "")
    hosp_dict["on_vent"]=hosp_table.iloc[3][1].replace(",", "")
    hosp_list.append(hosp_dict)
latest_hosp_df=pd.DataFrame(hosp_list)
latest_hosp_df.date=pd.to_datetime(latest_hosp_df.date, format='%m%d%y')
latest_hosp_df[['hospitalized', 'in_icu', 'on_vent']]=latest_hosp_df[['hospitalized', 'in_icu', 'on_vent']].apply(pd.to_numeric, errors = 'coerce')

final_hosp_df=pd.concat([hosp_df,latest_hosp_df]).reset_index(drop=True)
final_hosp_df=final_hosp_df.drop_duplicates()
final_hosp_df.date=pd.to_datetime(final_hosp_df.date, format='%Y-%m-%d')
final_hosp_df=final_hosp_df.sort_values('date', ascending=False).reset_index(drop=True)
latest_hosp_df

,date,hospitalized,in_icu,on_vent
0,2021-08-10,1251,339,168
1,2021-08-09,1139,331,158
2,2021-08-08,1083,321,138
3,2021-08-07,1054,311,130
